In [1]:
!nvidia-smi

Wed May 12 09:48:48 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   59C    P8    11W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


# Imports

In [3]:
!pip install pytorch-lightning

In [4]:
import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset
from enum import Enum
import torch.nn.functional as F

import numpy as np
import csv
import sys
import os
import random
import matplotlib.pyplot as plt
import matplotlib
import time
from math import floor
import tqdm
import random

import pytorch_lightning as pl

seed = 0

random.seed(seed)
torch.manual_seed(seed)
np.random.seed(seed)

# Hyper-parameters

In [5]:
data_dir = '/gdrive/MyDrive/BTP/Data/baseline_data/S08'
test_dir = '/gdrive/MyDrive/BTP/Data/baseline_data/S09'

BATCH_SIZE = 32
n_worker = 2

save_dir = '/gdrive/MyDrive/BTP/Models'
os.makedirs(save_dir,exist_ok=True)

color_space = 1

thresholdBelowMax = 6

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


# Utilities

## top k acc

In [6]:
def top_k_acc(y_true:torch.Tensor,y_pred:torch.Tensor,k=1):
    
    y_pred_tpk = torch.topk(y_pred,k,dim=1)[1]
    
    ovr = 0
    pos = 0

    for i in range(0,len(y_pred_tpk)):
        if(y_true[i] in y_pred_tpk[i]):
            pos+=1
        ovr+=1
    
    acc = pos/ovr
    return acc

In [7]:
def top_k_acc_tie(y_true:torch.Tensor,y_pred:torch.Tensor,k=1):
    pos=0
    ovr=0
    y_pred = y_pred.cpu().detach().numpy()
    y_true = y_true.cpu().detach().numpy()
    y_pred_idx = np.argsort(y_pred)[:,::-1]
    y_pred_val = np.sort(y_pred)[:,::-1]
    for i in range(0,y_pred.shape[0]):
        fin = list()
        idx = y_pred_idx[i]
        val = y_pred_val[i][k-1]
        for j in range(0,len(y_pred_val[i])):
            if(y_pred_val[i][j]>=val):
                fin.append(idx[j])
        if(y_true[i] in fin):
            pos+=1
        ovr+=1
    
    return pos/ovr


## Balancing data

In [8]:
def balance_data(y,coord,img,lidar,variance,dim1,dim2,dim3):
    Apperance = {i:np.count_nonzero(y == i) for i in y}
    Max_apperance = max(Apperance.values())
    
    for i in (Apperance.keys()):
        ind = [ind for ind, value in enumerate(y) if value == i]
        
        randperm = np.random.RandomState(seed).permutation(int(Max_apperance-Apperance[i]))%len(ind)
        
        ADD_beam = np.empty((len(randperm)))
        ADD_coord = np.empty((len(randperm),)+dim1)
        ADD_img = np.empty((len(randperm),)+dim2)
        ADD_lidar = np.empty((len(randperm),)+dim3)
        print('shapes',ADD_beam.shape)

        for couter,v in enumerate(randperm):
            ADD_beam[couter] = y[ind[v]]
            ADD_coord[couter,:] = coord[ind[v]]+variance*np.random.rand(*dim1)
            ADD_img[couter,:] = img[ind[v]]+variance*np.random.rand(*dim2)
            ADD_lidar[couter,:] = lidar[ind[v]]+variance*np.random.rand(*dim3)

        beams = np.concatenate((y, ADD_beam), axis=0)
        coord = np.concatenate((coord, ADD_coord), axis=0)
        img = np.concatenate((img, ADD_img), axis=0)
        lidar = np.concatenate((lidar, ADD_lidar), axis=0)

        return beams,coord,img,lidar



# Ranking beam output

Reading beam_output and converting them to logarithm scale; Power (dB) (neg) and setting a threshold value below which rounding all to zero

Overall classes: 8*32 = 256

In [9]:
def beamsLogScale(y:np,thresholdBelowMax):
    y_shape = y.shape
    
    for i in range(0,y_shape[0]):            
        thisOutputs = y[i,:]
        logOut = 20*np.log10(thisOutputs + 1e-30)
        minValue = np.amax(logOut) - thresholdBelowMax
        zeroedValueIndices = logOut < minValue
        thisOutputs[zeroedValueIndices]=0
        thisOutputs = thisOutputs / sum(thisOutputs)
        y[i,:] = thisOutputs
    
    return y

In [10]:
def getBeamOutput(output_file:str):
    
    print("Reading dataset...", output_file)
    output_cache_file = np.load(output_file)
    yMatrix = output_cache_file['output_classification']
    
    yMatrix = np.abs(yMatrix)
    yMatrix /= np.max(yMatrix)
    yMatrixShape = yMatrix.shape
    num_classes = yMatrix.shape[1] * yMatrix.shape[2] #Overall 
    
    y = yMatrix.reshape(yMatrix.shape[0],num_classes)
    y = beamsLogScale(y,thresholdBelowMax)
    
    return y,num_classes

# Creating Dataset and Dataloader


## Data loading for baseline

In [11]:
# coordURL = os.path.join(data_dir,'coord_input')
# imgURL = os.path.join(data_dir,'image_input')
# lidarURL = os.path.join(data_dir,'lidar_input')
# beamURL = os.path.join(data_dir,'beam_output')

# coordURL_t = os.path.join(test_dir,'coord_input')
# imgURL_t = os.path.join(test_dir,'image_input')
# lidarURL_t = os.path.join(test_dir,'lidar_input')
# beamURL_t = os.path.join(test_dir,'beam_output')

In [12]:
# coord_train = np.load(coordURL+'/coord_train.npz')['coordinates']
# coord_dev = np.load(coordURL+'/coord_validation.npz')['coordinates']

# img_train = np.load(imgURL+'/img_input_train_20.npz')['inputs']
# img_dev = np.load(imgURL+'/img_input_validation_20.npz')['inputs']

# lidar_train = np.load(lidarURL+'/lidar_train.npz')['input']
# lidar_dev = np.load(lidarURL+'/lidar_validation.npz')['input']

# coord_test = np.load(coordURL_t+'/coord_test.npz')['coordinates']

# img_test = np.load(imgURL_t+'/img_input_test_20.npz')['inputs']

# lidar_test = np.load(lidarURL_t+'/lidar_test.npz')['input']

In [13]:
# y_train,num_classes = getBeamOutput(beamURL+'/beams_output_train.npz')
# y_dev,_ = getBeamOutput(beamURL+'/beams_output_validation.npz')
# y_test,_ = getBeamOutput(beamURL_t+'/beams_output_test.npz')
# y_train = np.argmax(y_train,axis=1)
# y_dev = np.argmax(y_dev,axis=1)
# y_test = np.argmax(y_test,axis=1)
# print(num_classes)

## Balanced dataset creation

In [14]:
# data_b = data_dir+'_balanced'
# os.makedirs(data_b,exist_ok = True)
# os.makedirs(data_b+'/coord_input',exist_ok = True)
# os.makedirs(data_b+'/lidar_input',exist_ok = True)
# os.makedirs(data_b+'/image_input',exist_ok = True)
# os.makedirs(data_b+'/beam_output',exist_ok = True)

In [15]:
# y_train,coord_train,img_train,lidar_train = balance_data(y_train,coord_train,img_train,lidar_train,
#                                                          0.001,
#                                                          (2,),(48,81,1),(20,200,10)
#                                                          )
# print(y_train.shape,coord_train.shape,img_train.shape,lidar_train.shape)

# np.savez(data_b+'/beam_output/beam_output_train.npz',output_classification = y_train)
# np.savez(data_b+'/coord_input/coord_train.npz',coordinates = coord_train)
# np.savez(data_b+'/image_input/img_input_train_20.npz',inputs = img_train)
# np.savez(data_b+'/lidar_input/lidar_train.npz',inputs = lidar_train)

In [16]:
# y_dev,coord_dev,img_dev,lidar_dev = balance_data(y_dev,coord_dev,img_dev,lidar_dev,
#                                                          0.001,
#                                                          (2,),(48,81,1),(20,200,10)
#                                                          )
# print(coord_dev.shape,img_dev.shape,lidar_dev.shape)

# np.savez(data_b+'/beam_output/beam_output_val.npz',output_classification = y_dev)
# np.savez(data_b+'/coord_input/coord_val.npz',coordinates = coord_dev)
# np.savez(data_b+'/image_input/img_input_val_20.npz',inputs = img_dev)
# np.savez(data_b+'/lidar_input/lidar_val.npz',inputs = lidar_dev)

## Loading balanced dataset

In [17]:
data_dir = data_dir + '_balanced'

In [18]:
coordURL = os.path.join(data_dir,'coord_input')
imgURL = os.path.join(data_dir,'image_input')
lidarURL = os.path.join(data_dir,'lidar_input')
beamURL = os.path.join(data_dir,'beam_output')

In [19]:
coordURL_t = os.path.join(test_dir,'coord_input')
imgURL_t = os.path.join(test_dir,'image_input')
lidarURL_t = os.path.join(test_dir,'lidar_input')
beamURL_t = os.path.join(test_dir,'beam_output')

In [20]:
coord_train = np.load(coordURL+'/coord_train.npz')['coordinates']
coord_dev = np.load(coordURL+'/coord_val.npz')['coordinates']
coord_comb = np.concatenate((coord_train,coord_dev),axis=0)

img_train = np.load(imgURL+'/img_input_train_20.npz')['inputs']
img_dev = np.load(imgURL+'/img_input_val_20.npz')['inputs']
img_comb = np.concatenate((img_train,img_dev),axis=0)

lidar_train = np.load(lidarURL+'/lidar_train.npz')['inputs']
lidar_dev = np.load(lidarURL+'/lidar_val.npz')['inputs']
# lidar_comb = np.concatenate((lidar_train,lidar_dev),axis=0)

In [21]:
coord_test = np.load(coordURL_t+'/coord_test.npz')['coordinates']

img_test = np.load(imgURL_t+'/img_input_test_20.npz')['inputs']

lidar_test = np.load(lidarURL_t+'/lidar_test.npz')['input']

In [22]:
y_train = np.load(data_dir+'/beam_output/beam_output_train.npz')['output_classification']
y_dev = np.load(data_dir+'/beam_output/beam_output_val.npz')['output_classification']
# y_comb = np.concatenate((y_train,y_dev),axis=0)

y_test,num_classes = getBeamOutput(beamURL_t+'/beams_output_test.npz')
y_test = np.argmax(y_test,axis=1)
print(num_classes)

Reading dataset... /gdrive/MyDrive/BTP/Data/baseline_data/S09/beam_output/beams_output_test.npz
256


## Creating Dataset and Dataloader

### Custom Dataset

In [23]:
from PIL import Image
from torchvision import transforms
transform_pipe = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [24]:
class custom_dataset(Dataset):
    def __init__(self,coord,img,lidar,label,transform=None):
        self.coord = coord
        self.img = img
        self.lidar = lidar
        self.label =  label
        self.transform = transform
    
    def __getitem__(self,idx):
        sample = dict()

        sample['coord'] = self.coord[idx]
        sample['lidar'] = self.lidar[idx].transpose(2,0,1)
        sample['label'] = self.label[idx]
        
        img_sample =  self.img[idx].transpose(2,0,1)[::-1,:,:].copy()

        a,b,c = (img_sample.shape)
        c = int(c/3)

        img1 = img_sample[:,:,0:c]
        img2 = img_sample[:,:,c:2*c]
        img3 = img_sample[:,:,2*c:3*c]

        sample['img1'] = img1
        sample['img2'] = img2
        sample['img3'] = img3

        return sample
    
    def __len__(self):
        return len(self.label)
 

In [25]:
train_dataset = custom_dataset(coord_train,
                               img_train,
                               lidar_train,
                               y_train)

In [26]:
dev_dataset = custom_dataset(coord_dev,
                             img_dev,
                             lidar_dev,
                             y_dev)

In [27]:
test_dataset = custom_dataset(coord_test,
                             img_test,
                             lidar_test,
                             y_test)

In [28]:
# comb_dataset = custom_dataset(coord_comb,
#                               img_comb,
#                               lidar_comb,
#                               y_comb)

In [29]:
print(len(train_dataset))
print(len(dev_dataset))
print(len(test_dataset))
# print(len(comb_dataset))

9706
2265
9638


### Dataloader

In [30]:
train_loader = DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    pin_memory=True,
    num_workers = n_worker,
    drop_last = True,
    shuffle = True
    )

In [31]:
dev_loader = DataLoader(
    dev_dataset,
    batch_size=BATCH_SIZE,
    pin_memory=True,
    num_workers = n_worker,
    drop_last = True,
    )

In [32]:
test_loader = DataLoader(
    test_dataset,
    batch_size=BATCH_SIZE,
    pin_memory=True,
    num_workers = n_worker,
    drop_last = True,
    )

In [33]:
# comb_loader = DataLoader(
#     comb_dataset,
#     batch_size=BATCH_SIZE,
#     pin_memory=True,
#     num_workers = n_worker,
#     drop_last = True,
#     shuffle = True
#     )

# Coord_mlp

## Model Class

In [34]:
class LitModel(pl.LightningModule):
    def __init__(self,learning_rate):
        super().__init__()
        
        self.CEloss = nn.CrossEntropyLoss()
        self.learning_rate = learning_rate

        self.conv1 = nn.Sequential(nn.Conv1d(1,20,2),nn.ReLU())
        self.conv2 = nn.Sequential(nn.Conv1d(20,10,2,padding = 1),nn.ReLU())
        self.mpool = nn.MaxPool1d(2)

        self.conv3 = nn.Sequential(nn.Conv1d(10,20,2,padding=1),nn.ReLU())
        self.conv4 = nn.Sequential(nn.Conv1d(20,10,2,padding=1),nn.ReLU())

        self.flatten = nn.Flatten()

        self.fc = nn.Sequential(nn.Linear(10,1024),nn.ReLU(),
                                  nn.Linear(1024,512),nn.ReLU(),
                                  nn.Linear(512,256))


    def forward(self,X1):
        X1 = self.conv1(X1)
        X1 = self.conv2(X1)
        X1 = self.mpool(X1)

        X1 = self.conv3(X1)
        X1 = self.conv4(X1)
        X1 = self.mpool(X1)

        out = self.flatten(X1)
        out = self.fc(out)
        return out

    
    def training_step(self,batch,batch_idx):
        coord = batch['coord'].float().reshape((BATCH_SIZE,1,2))
        label = batch['label']
        y_hat = self(coord)
        loss = self.CEloss(y_hat,label)
        self.log('my_loss',loss)
        return {'loss':loss,'pred':y_hat,'label':label}

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr= self.learning_rate)
        return optimizer

    def training_epoch_end(self,train_out):
        len_out = len(train_out)
        y_pred = torch.Tensor(len_out*BATCH_SIZE,num_classes)
        y_true = torch.Tensor(len_out*BATCH_SIZE)

        for i in range(0,len_out):
            y_pred[i*BATCH_SIZE:(i+1)*BATCH_SIZE,:] = train_out[i]['pred'] 
            y_true[i*BATCH_SIZE:(i+1)*BATCH_SIZE] = train_out[i]['label']

        top1 = top_k_acc(y_true,y_pred,k=1)
        top5 = top_k_acc(y_true,y_pred,k=5)
        top10 = top_k_acc(y_true,y_pred,k=10)
        top50 = top_k_acc(y_true,y_pred,k=50)

        print('Train Topk accuracies are :  Top 1: {:.4f}, Top 5: {:.4f}, Top 10: {:.4f},Top 50: {:.4f}'.format(top1,top5,top10,top50))


    def validation_step(self,batch,batch_idx):
        coord = batch['coord'].float().reshape((BATCH_SIZE,1,2))
        label = batch['label']
        yhat = self(coord)
        return [yhat.cpu().detach(),label.cpu().detach()]
    
    def validation_epoch_end(self,val_out):
        len_out = len(val_out)
        y_pred = torch.Tensor(len_out*BATCH_SIZE,num_classes)
        y_true = torch.Tensor(len_out*BATCH_SIZE)

        for i in range(0,len_out):
            y_pred[i*BATCH_SIZE:(i+1)*BATCH_SIZE,:] = val_out[i][0] 
            y_true[i*BATCH_SIZE:(i+1)*BATCH_SIZE] = val_out[i][1] 

        top1 = top_k_acc(y_true,y_pred,k=1)
        top5 = top_k_acc(y_true,y_pred,k=5)
        top10 = top_k_acc(y_true,y_pred,k=10)
        top50 = top_k_acc(y_true,y_pred,k=50)

        print('Dev Topk accuracies are :  Top 1: {:.4f}, Top 5: {:.4f}, Top 10: {:.4f},Top 50: {:.4f}'.format(top1,top5,top10,top50))

    def test_step(self,batch,batch_idx):
        label = batch['label']
        coord = batch['coord'].float().reshape((BATCH_SIZE,1,2))

        yhat = self(coord)

        return [yhat.cpu().detach(),label.cpu().detach()]
    
    def test_epoch_end(self,test_out):
        len_out = len(test_out)
        y_pred = torch.Tensor(len_out*BATCH_SIZE,num_classes)
        y_true = torch.Tensor(len_out*BATCH_SIZE)

        for i in range(0,len_out):
            y_pred[i*BATCH_SIZE:(i+1)*BATCH_SIZE,:] = test_out[i][0] 
            y_true[i*BATCH_SIZE:(i+1)*BATCH_SIZE] = test_out[i][1] 

        top1 = top_k_acc(y_true,y_pred,k=1)
        top5 = top_k_acc(y_true,y_pred,k=5)
        top10 = top_k_acc(y_true,y_pred,k=10)
        top50 = top_k_acc(y_true,y_pred,k=50)
        print('Test Topk accuracies are :  Top 1: {:.4f}, Top 5: {:.4f}, Top 10: {:.4f},Top 50: {:.4f}'.format(top1,top5,top10,top50))

## Running Trainer

In [35]:
coord_litmodel = LitModel(learning_rate=1e-4)

In [36]:
# trainer_coord = pl.Trainer(reload_dataloaders_every_epoch = True,
#                             auto_lr_find = True,
#                             gpus=1,
#                             max_epochs = 30,
#                             num_sanity_val_steps=-1
#                             )

GPU available: True, used: True
TPU available: False, using: 0 TPU cores


In [37]:
# trainer_coord.tune(coord_litmodel,train_loader,dev_loader)
# trainer_coord.fit(coord_litmodel,train_loader,dev_loader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name    | Type             | Params
---------------------------------------------
0 | CEloss  | CrossEntropyLoss | 0     
1 | conv1   | Sequential       | 60    
2 | conv2   | Sequential       | 410   
3 | mpool   | MaxPool1d        | 0     
4 | conv3   | Sequential       | 420   
5 | conv4   | Sequential       | 410   
6 | flatten | Flatten          | 0     
7 | fc      | Sequential       | 667 K 
---------------------------------------------
668 K     Trainable params
0         Non-trainable params
668 K     Total params
2.675     Total estimated model params size (MB)


Dev Topk accuracies are :  Top 1: 0.0000, Top 5: 0.0121, Top 10: 0.0638,Top 50: 0.1540


RuntimeError: ignored

In [ ]:
# trainer_coord.test(coord_litmodel,test_loader)

# Model Class: light_image

In [ ]:
class light_image(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(color_space,8,(3,3),padding=0)
        self.mpool = nn.MaxPool2d((2,2),stride = (1,1))
        self.conv2a = nn.Conv2d(8,8,(3,3))
        self.conv2b = nn.Conv2d(8,8,(3,3))
        self.relu = nn.ReLU()
        self.drop = torch.nn.Dropout(p=0.25)
        self.flatten = nn.Flatten()
        self.fc3 = nn.Linear(12480,256)

    def forward(self,X1,X2,X3):
        b = int(X1.shape[3]/3)
        
        t1 = self.conv_forward(X1)
        t2 = self.conv_forward(X2)
        t3 = self.conv_forward(X3)
        t = torch.cat([t1,t2,t3],dim=1)
        # t = self.fc1(t)
        # t = self.fc2(t)
        t  = self.fc3(t)

        return t


    def conv_forward(self,X1):
        X1 = self.conv1(X1)
        X1 = self.relu(X1)
        X1 = self.mpool(X1)

        X1 = self.conv2a(X1)
        X1 = self.relu(X1)
        X1 = self.mpool(X1)

        X1 = self.conv2b(X1)
        X1 = self.relu(X1)

        X1 = self.drop(X1)
        X1 = self.flatten(X1)
        
        return X1


### Intializing Model

In [ ]:
# def init_weights_img(m):
#     if type(m) == (nn.Linear or nn.Conv2d):
#         torch.nn.init.xavier_uniform_(m.weight)
#         m.bias.data.fill_(0.01)  

In [ ]:
# model_img = light_image()
# model_img = model_img.apply(init_weights_img)

# Lidar

## Lightning Class

In [53]:
class lidar_lit(pl.LightningModule):
    def __init__(self,learning_rate = 1e-3):
        super().__init__()

        self.drop_prob = 0.3
        self.drop = nn.Dropout(self.drop_prob)
        
        self.conv0 = nn.Sequential(nn.Conv2d(10,32,(3,3)),nn.ReLU())
        self.conv1 = nn.Sequential(nn.Conv2d(32,32,(3,3)),nn.ReLU())
        self.conv2 = nn.Sequential(nn.Conv2d(32,32,(3,3)),nn.ReLU())
        self.conv3 = nn.Sequential(nn.Conv2d(32,32,(3,3)),nn.ReLU())
        self.conv4 = nn.Sequential(nn.Conv2d(32,32,(3,3)),nn.ReLU())
        self.conv5 = nn.Sequential(nn.Conv2d(32,32,(3,3)),nn.ReLU())
        self.conv6 = nn.Sequential(nn.Conv2d(32,32,(3,3)),nn.ReLU())
        self.conv7 = nn.Sequential(nn.Conv2d(32,32,(3,3)),nn.ReLU())
        self.conv8 = nn.Sequential(nn.Conv2d(32,32,(3,3)),nn.ReLU())

        self.m_pool1 = nn.MaxPool2d((2,2))
        self.m_pool2 = nn.MaxPool2d((1,2))

        self.flatten = nn.Flatten()

        self.drop_prob_fc = 0.2
        self.drop_fc = nn.Dropout(self.drop_prob_fc)
        self.fc = nn.Sequential(nn.Linear(4000,512),
                                nn.ReLU(),
                                self.drop_fc,
                                nn.Linear(512,256),
                                nn.ReLU(),
                                self.drop_fc,
                                nn.Linear(256,256)
                                )

        self.CEloss = nn.CrossEntropyLoss()
        self.learning_rate = learning_rate
        self.L1 = nn.L1Loss(reduction='sum')
        self.L1_fac = 1e-5
        self.L2_fac = 1e-4

            self.soft = nn.Softmax(dim=1)

    def forward(self,X):
        
        a = X = self.conv0(X)
        X = F.pad(X,(1,1,1,1))
        a = F.pad(a,(1,1,1,1))
        X = self.conv1(X)
        X = F.pad(X,(1,1,1,1))
        X = self.conv2(X)
        X = F.pad(X,(1,1,1,1))
        X = X+a
        X = self.m_pool1(X)


        b = X= self.drop(X)

        X = self.conv3(X)
        X = F.pad(X,(1,1,1,1))
        X = self.conv4(X)
        X = F.pad(X,(1,1,1,1))
        X = X+b
        X = self.m_pool1(X)

        c = X = self.drop(X)

        X = self.conv5(X)
        X = F.pad(X,(1,1,1,1))
        X = self.conv6(X)
        X = F.pad(X,(1,1,1,1))
        X = X + c
        X = self.m_pool2(X)
        
        d = X = self.drop(X)

        X = self.conv7(X)
        X = F.pad(X,(1,1,1,1))
        X = self.conv8(X)
        X = F.pad(X,(1,1,1,1))
        X = X + d

        out = self.flatten(X)
        out = self.fc(out)
        
        return out

    def add_l1_l2_reg(self):
        # l2_reg = self.L2_fac*(torch.linalg.norm(self.fc[0].weight)+torch.linalg.norm(self.fc[3].weight)).float()
        l1_reg = self.L1_fac*(torch.linalg.norm(self.fc[0].weight,ord=1)+torch.linalg.norm(self.fc[3].weight,ord=1)).float()

        return l1_reg 

    def training_step(self,batch,batch_idx):
        label = batch['label'].long()
        lidar = batch['lidar'].float()

        yhat = self(lidar)

        loss = self.CEloss(yhat,label)
        
        loss += self.add_l1_l2_reg()
        L1_loss = self.L1(yhat,F.one_hot(label.long(),num_classes))*self.L1_fac

        loss += L1_loss
        self.log('my_loss',loss)

        return {'loss':loss,'pred':yhat,'label':label}

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr= self.learning_rate,weight_decay=1e-4)
        return optimizer

    def training_epoch_end(self,train_out):
        len_out = len(train_out)
        y_pred = torch.Tensor(len_out*BATCH_SIZE,num_classes)
        y_true = torch.Tensor(len_out*BATCH_SIZE)

        for i in range(0,len_out):
            y_pred[i*BATCH_SIZE:(i+1)*BATCH_SIZE,:] = train_out[i]['pred'] 
            y_true[i*BATCH_SIZE:(i+1)*BATCH_SIZE] = train_out[i]['label']

        top1 = top_k_acc(y_true,y_pred,k=1)
        top5 = top_k_acc(y_true,y_pred,k=5)
        top10 = top_k_acc(y_true,y_pred,k=10)
        top50 = top_k_acc(y_true,y_pred,k=50)

        print('Train Topk accuracies are :  Top 1: {:.4f}, Top 5: {:.4f}, Top 10: {:.4f},Top 50: {:.4f}'.format(top1,top5,top10,top50))

    def validation_step(self,batch,batch_idx):
        label = batch['label'].long()
        lidar = batch['lidar'].float()

        yhat = self(lidar)

        return [yhat.cpu().detach(),label.cpu().detach()]
    
    def validation_epoch_end(self,val_out):
        len_out = len(val_out)
        y_pred = torch.Tensor(len_out*BATCH_SIZE,num_classes)
        y_true = torch.Tensor(len_out*BATCH_SIZE)

        for i in range(0,len_out):
            y_pred[i*BATCH_SIZE:(i+1)*BATCH_SIZE,:] = val_out[i][0] 
            y_true[i*BATCH_SIZE:(i+1)*BATCH_SIZE] = val_out[i][1] 

        top1 = top_k_acc(y_true,y_pred,k=1)
        top5 = top_k_acc(y_true,y_pred,k=5)
        top10 = top_k_acc(y_true,y_pred,k=10)
        top50 = top_k_acc(y_true,y_pred,k=50)

        print('Dev Topk accuracies are :  Top 1: {:.4f}, Top 5: {:.4f}, Top 10: {:.4f},Top 50: {:.4f}'.format(top1,top5,top10,top50))

    def test_step(self,batch,batch_idx):
        label = batch['label'].long()
        lidar = batch['lidar'].float()

        yhat = self(lidar)

        return [yhat.cpu().detach(),label.cpu().detach()]
    
    def test_epoch_end(self,test_out):
        len_out = len(test_out)
        y_pred = torch.Tensor(len_out*BATCH_SIZE,num_classes)
        y_true = torch.Tensor(len_out*BATCH_SIZE)

        for i in range(0,len_out):
            y_pred[i*BATCH_SIZE:(i+1)*BATCH_SIZE,:] = test_out[i][0] 
            y_true[i*BATCH_SIZE:(i+1)*BATCH_SIZE] = test_out[i][1] 

        top1 = top_k_acc(y_true,y_pred,k=1)
        top5 = top_k_acc(y_true,y_pred,k=5)
        top10 = top_k_acc(y_true,y_pred,k=10)
        top50 = top_k_acc(y_true,y_pred,k=50)

        print('Test Topk accuracies are :  Top 1: {:.4f}, Top 5: {:.4f}, Top 10: {:.4f},Top 50: {:.4f}'.format(top1,top5,top10,top50))

## Runnning Trainer

In [54]:
# def init_weights_lid(m):
#     if type(m) == (nn.Linear or nn.Conv2d):
#         torch.nn.init.xavier_uniform_(m.weight)
#         m.bias.data.fill_(0.000)  

In [55]:
model_lidar = lidar_lit(1e-4)
# model_lidar = model_lidar.apply(init_weights_lid)

In [56]:
trainer_lidar = pl.Trainer(reload_dataloaders_every_epoch = True,
                     gpus=1,
                     max_epochs = 50,
                     num_sanity_val_steps=-1,
                     auto_lr_find = False
                     )

GPU available: True, used: True
TPU available: False, using: 0 TPU cores


In [57]:
# trainer_lidar.tune(model_lidar,train_loader,dev_loader)

In [58]:
# trainer_lidar.fit(model_lidar,comb_loader,test_loader)

In [59]:
trainer_lidar.fit(model_lidar,train_loader,dev_loader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

   | Name    | Type             | Params
----------------------------------------------
0  | drop    | Dropout          | 0     
1  | conv0   | Sequential       | 2.9 K 
2  | conv1   | Sequential       | 9.2 K 
3  | conv2   | Sequential       | 9.2 K 
4  | conv3   | Sequential       | 9.2 K 
5  | conv4   | Sequential       | 9.2 K 
6  | conv5   | Sequential       | 9.2 K 
7  | conv6   | Sequential       | 9.2 K 
8  | conv7   | Sequential       | 9.2 K 
9  | conv8   | Sequential       | 9.2 K 
10 | m_pool1 | MaxPool2d        | 0     
11 | m_pool2 | MaxPool2d        | 0     
12 | flatten | Flatten          | 0     
13 | drop_fc | Dropout          | 0     
14 | fc      | Sequential       | 2.2 M 
15 | CEloss  | CrossEntropyLoss | 0     
16 | L1      | L1Loss           | 0     
17 | soft    | Softmax          | 0     
----------------------------------------------
2.3 M     Trainable params
0         Non-trainable params
2.3 M     Total params
9.2

Dev Topk accuracies are :  Top 1: 0.0000, Top 5: 0.0049, Top 10: 0.0143,Top 50: 0.1254


Train Topk accuracies are :  Top 1: 0.2131, Top 5: 0.5209, Top 10: 0.6643,Top 50: 0.9168


Dev Topk accuracies are :  Top 1: 0.2504, Top 5: 0.5817, Top 10: 0.7424,Top 50: 0.9790
Train Topk accuracies are :  Top 1: 0.2831, Top 5: 0.6422, Top 10: 0.7892,Top 50: 0.9746


Dev Topk accuracies are :  Top 1: 0.2670, Top 5: 0.6571, Top 10: 0.8071,Top 50: 0.9821
Train Topk accuracies are :  Top 1: 0.3094, Top 5: 0.6849, Top 10: 0.8169,Top 50: 0.9792


...

Dev Topk accuracies are :  Top 1: 0.6098, Top 5: 0.8960, Top 10: 0.9589,Top 50: 0.9839
Train Topk accuracies are :  Top 1: 0.7509, Top 5: 0.9749, Top 10: 0.9948,Top 50: 1.0000


Dev Topk accuracies are :  Top 1: 0.5808, Top 5: 0.9054, Top 10: 0.9371,Top 50: 0.9826
Train Topk accuracies are :  Top 1: 0.7495, Top 5: 0.9750, Top 10: 0.9959,Top 50: 0.9999


Dev Topk accuracies are :  Top 1: 0.5978, Top 5: 0.9004, Top 10: 0.9469,Top 50: 0.9835
Train Topk accuracies are :  Top 1: 0.7596, Top 5: 0.9771, Top 10: 0.9959,Top 50: 1.0000


Dev Topk accuracies are :  Top 1: 0.5835, Top 5: 0.9094, Top 10: 0.9603,Top 50: 0.9839
Train Topk accuracies are :  Top 1: 0.7628, Top 5: 0.9779, Top 10: 0.9970,Top 50: 1.0000


Dev Topk accuracies are :  Top 1: 0.5795, Top 5: 0.8884, Top 10: 0.9451,Top 50: 0.9826



In [60]:
trainer_lidar.test(model = model_lidar,test_dataloaders=test_loader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Test Topk accuracies are :  Top 1: 0.4386, Top 5: 0.7919, Top 10: 0.8738,Top 50: 0.9616

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{}
--------------------------------------------------------------------------------


[{}]

# Resnet

## Model Class

In [ ]:
import torchvision.models as models

res18 = models.resnet18(pretrained = True)

for params in res18.parameters():
    params.requires_grad = False

In [ ]:
class res18_lit(pl.LightningModule):
    def __init__(self,learning_rate = 1e-3):
        super().__init__()

        self.activation = {}

        self.resnet = res18
        self.resnet.avgpool.register_forward_hook(self.get_res_activation('out',self.activation))
        self.fc1 = nn.Linear(512*3,1024)
        self.fc2 = nn.Linear(1024,512)
        self.fc3 = nn.Linear(512,256)

        self.CEloss = nn.CrossEntropyLoss()
        self.learning_rate = learning_rate

    def get_res_activation(self,name, activation):
        def hook(model, input, output):
            activation[name] = output.detach()
        return hook

    def forward(self,X1,X2,X3):
        res_out = self.resnet(X1)
        t1 = self.activation['out'].reshape((BATCH_SIZE,512))
        res_out = self.resnet(X2)
        t2 = self.activation['out'].reshape((BATCH_SIZE,512))
        res_out = self.resnet(X3)
        t3 = self.activation['out'].reshape((BATCH_SIZE,512))

        t = torch.cat([t1,t2,t3],dim = 1)

        out = self.fc1(t)
        out = self.fc2(out)
        out = self.fc3(out)

        return out
    
    def training_step(self,batch,batch_idx):
        label = batch['label']

        img1 = batch['img1'].float()
        img2 = batch['img2'].float()
        img3 = batch['img3'].float()

        y_hat = self(img1,img2,img3)

        loss = self.CEloss(y_hat,label)
        self.log('my_loss',loss)

        return {'loss':loss,'pred':y_hat,'label':label}

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr= self.learning_rate)
        return optimizer

    def training_epoch_end(self,train_out):
        len_out = len(train_out)
        y_pred = torch.Tensor(len_out*BATCH_SIZE,num_classes)
        y_true = torch.Tensor(len_out*BATCH_SIZE)

        for i in range(0,len_out):
            y_pred[i*BATCH_SIZE:(i+1)*BATCH_SIZE,:] = train_out[i]['pred'] 
            y_true[i*BATCH_SIZE:(i+1)*BATCH_SIZE] = train_out[i]['label']

        top1 = top_k_acc(y_true,y_pred,k=1)
        top5 = top_k_acc(y_true,y_pred,k=5)
        top10 = top_k_acc(y_true,y_pred,k=10)
        top50 = top_k_acc(y_true,y_pred,k=50)

        print(f'Train Topk accuracies are :  Top 1: {top1}, Top 5: {top5}, Top 10: {top10},Top 50 {top50}')


    def validation_step(self,batch,batch_idx):
        label = batch['label']

        img1 = batch['img1'].float()
        img2 = batch['img2'].float()
        img3 = batch['img3'].float()

        yhat = self(img1,img2,img3)

        return [yhat.cpu().detach(),label.cpu().detach()]
    
    def validation_epoch_end(self,val_out):
        len_out = len(val_out)
        y_pred = torch.Tensor(len_out*BATCH_SIZE,num_classes)
        y_true = torch.Tensor(len_out*BATCH_SIZE)

        for i in range(0,len_out):
            y_pred[i*BATCH_SIZE:(i+1)*BATCH_SIZE,:] = val_out[i][0] 
            y_true[i*BATCH_SIZE:(i+1)*BATCH_SIZE] = val_out[i][1] 

        top1 = top_k_acc(y_true,y_pred,k=1)
        top5 = top_k_acc(y_true,y_pred,k=5)
        top10 = top_k_acc(y_true,y_pred,k=10)
        top50 = top_k_acc(y_true,y_pred,k=50)

        print(f'Dev Topk accuracies are :  Top 1: {top1}, Top 5: {top5}, Top 10: {top10},Top 50 {top50}')

## Running Model

In [ ]:
# model_res = res18_lit()

In [ ]:
# sum(p.numel() for p in model_res.parameters() if p.requires_grad)

In [ ]:
# trainer_img = pl.Trainer(reload_dataloaders_every_epoch = True,
#                      gpus=1,
#                      max_epochs = 10,
#                      num_sanity_val_steps=-1,
#                      auto_lr_find = True
#                      )

In [ ]:
# trainer_img.tune(model_res,train_loader)
# trainer_img.fit(model_res,train_loader,dev_loader)